# Monkey patching

Motivations :
- changer le comportement d'une bibliothèque existante (pas toujours une bonne idée, mais parfois bien pratique)
- remplacer une fonction ou un objet par une « doublure » dans un test
    - résultat non déterministe
    - accès à une ressource extérieure
    - traitement particulièrement long

In [1]:
#! pip install requests

In [10]:
from xml.etree import ElementTree as ET

import requests


def actualité_de_la_psf():
    resp = requests.get("http://feeds.feedburner.com/PythonSoftwareFoundationNews")
    tree = ET.fromstring(resp.text)
    return [
        node.find("{http://www.w3.org/2005/Atom}title").text
        for node in tree.findall("{http://www.w3.org/2005/Atom}entry")
    ]


actualité_de_la_psf()

['Mozilla and Chan Zuckerberg Initiative to support pip',
 'Giving Tuesday 2019',
 'Python Software Foundation Fellow Members for Q3 2019',
 'Why Sponsor PyCon 2020?',
 'Seeking Developers for Paid Contract Improving pip',
 'The 2019 Python Developer Survey is here, take a few minutes to complete the survey!',
 'CPython Core Developer Sprint 2019',
 'Chris Angelico: 2019 Q2 Community Service Award Winner',
 'Grants Awarded for Python in Education ',
 'Felipe de Morais: 2019 Q2 Community Service Award Winner',
 'PyPI Security Q4 2019 Request for Proposals period opens.',
 'The Python Software Foundation has updated its Code of Conduct',
 'PyPI Security Q4 2019 Request for Information period opens.',
 'Python Software Foundation Fellow Members for Q1 & Q2 2019',
 'Humble Bundle by No Starch supports the Python Software Foundation!',
 'PyPI now supports uploading via API token',
 '2019 PSF Fundraiser - Thank you & debrief  ',
 'The Python Software Foundation is looking for bloggers!',
 'P

## Patcher à la main

In [11]:
EXEMPLE = """\
<?xml version="1.0" encoding="UTF-8"?>
<feed xmlns="http://www.w3.org/2005/Atom">
  <entry>
    <title type="text">Mozilla and Chan Zuckerberg Initiative to support pip</title>
  </entry>
  <entry>
    <title type="text">Giving Tuesday 2019</title>
  </entry>
</feed>
"""

In [12]:
class FausseRéponse:
    def __init__(self, text):
        self.text = text

In [13]:
def _get_doublure(*args, **kwargs):
    return FausseRéponse(EXEMPLE)

In [21]:
import requests


def test_actualité_de_la_psf():
    _get_original = requests.get  # garder de côté la fonction originale
    requests.get = _get_doublure  # la remplacer par notre doublure
    try:
        titres = actualité_de_la_psf()
    finally:
        requests.get = _get_original  # remettre en place la fonction originale

    assert titres == [
        "Mozilla and Chan Zuckerberg Initiative to support pip",
        "Giving Tuesday 2019",
    ]
    
test_actualité_de_la_psf()

## Patcher avec `patch`

In [19]:
from unittest.mock import patch


def test_actualité_de_la_psf():
    with patch("requests.get", _get_doublure):
        titres = actualité_de_la_psf()

    assert titres == [
        "Mozilla and Chan Zuckerberg Initiative to support pip",
        "Giving Tuesday 2019",
    ]

test_actualité_de_la_psf()

## Références

- https://docs.python.org/3/library/unittest.mock.html#the-patchers